In [3]:
import pandas as pd
import json, os
from tqdm import tqdm

path = "/Users/DToma/data/Rhenus_StartingFrom_20240101_20240701"

file = "response_07-2024.json"

data = []
for file in tqdm(os.listdir(path)):
  if file.endswith(".json"):
    with open(os.path.join(path, file), "r") as f:
        data.extend(json.load(f))
print(len(data))
data = pd.DataFrame(data)

100%|██████████| 7/7 [00:02<00:00,  3.13it/s]


140321


In [4]:
data.columns

Index(['TicketNr', 'TicketSummary', 'TicketDescription', 'CreatedAt',
       'CreatedByID', 'CreatedByString', 'ImpactedUser', 'ImpactedUserString',
       'AssignedToID', 'AssignedToString', 'RecipientRoleID',
       'RecipientRoleString', 'ImpactValue', 'ImpactString', 'UrgencyValue',
       'UrgencyString', 'PriorityValue', 'PriorityString', 'StatusValue',
       'StatusString', 'EntryByValue', 'EntryByString', 'ServiceAffected',
       'ServiceAffectedPortalName', 'AssetAffected', 'AssetAffectedName',
       'ServiceReported', 'ServiceReportedPortalName', 'AssetReported',
       'AssetReportedName', 'SLAID', 'SLAString', 'Type',
       'Expression-TypeCase', 'Expression-TypeID'],
      dtype='object')

In [5]:
cnt = 1 
for col in data.columns:
    # if col in ['SLAString', 'SLAID']:
    #     print(f"{col}: unique items: {data[col].nunique()} missing values percent: {data[col].isna().sum()*100/len(data)}%")
    # if col in ['TicketSummary', 'TicketDescription', 'CreatedAt','ServiceAffected','ImpactString','UrgencyString']:
    #     print(f"{col}: unique items: {data[col].nunique()} missing values percent: {data[col].isna().sum()*100/len(data)}%")
    # if col in ['EntryByValue', 'EntryByString','CreatedByID', 'CreatedByString', 'ImpactedUser', 'ImpactedUserString','AssignedToID', 'AssignedToString', 'RecipientRoleID','RecipientRoleString']:
    #     print(f"{col}: unique items: {data[col].nunique()} missing values percent: {data[col].isna().sum()*100/len(data)}%")

    if col in [ 'ServiceAffected','ServiceAffectedPortalName','ServiceReported', 'ServiceReportedPortalName']:
        if cnt:
            print("\n")
            cnt = 0
        print(f"{col}: unique items: {data[col].nunique()} missing values percent: {data[col].isna().sum()*100/len(data)}%")



ServiceAffected: unique items: 1011 missing values percent: 9.998503431417964%
ServiceAffectedPortalName: unique items: 891 missing values percent: 42.672158835812176%
ServiceReported: unique items: 692 missing values percent: 3.3708425681116867%
ServiceReportedPortalName: unique items: 598 missing values percent: 37.897392407408724%


In [5]:
data['ServiceAffectedPortalName'].value_counts()[:10]

ServiceAffectedPortalName
RSD-IT-Support                  10245
SAP                              3647
Mail Service mit Lotus Notes     3621
FGER-LWIS                        3370
FGER-LOXX Standard               3025
RAS-SUPPORT                      2667
FEU-PL-TMS-General               2390
RML-Network                      2247
FEU-PL General                   1657
RSND-Einkauf                     1562
Name: count, dtype: int64

In [4]:
data['UrgencyString'].value_counts()

UrgencyString
C: Continued work is possible                     73747
B: Continued work is possible with limitations    63287
A: Continued work is not possible                  3287
Name: count, dtype: int64

In [5]:
data['TicketDescription'].value_counts()

TicketDescription
Please use an HTML-capable email client to correctly display this email. Alternatively, switch to 'Text' format in the notification settings.\r\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              3173
Empty E-mail Message                                                                                                                                                                                                              

In [7]:
data['TicketSummary'].value_counts()

TicketSummary
MATRIX PersonImport                                               936
[PRTG Network Monitor] 25 Summarized Notifications                860
[DEMHG01_D] Integrator Transaction Failure                        677
Sandbox: E-Mail-Vorgangserfassung: Fehler bei der Verarbeitung    648
[DENUE01_P] Background service failed                             614
                                                                 ... 
[PROBLEM] Host deulz01-apr057 is DOWN!                              1
[PROBLEM] Host deulz01-apr083 is DOWN!                              1
[PROBLEM] Host deulz01-apr093 is DOWN!                              1
[PROBLEM] Host deulz01-apr164 is DOWN!                              1
Asset Clearing WSS IT Team Mitte - Mon 06/24/2024                   1
Name: count, Length: 112686, dtype: int64

In [1]:
from sentence_transformers import SentenceTransformer


/Users/DToma/miniconda3/envs/pyml/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/Users/DToma/miniconda3/envs/pyml/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
tokens = model.tokenizer(['dddd','dddddsd'], padding=True, truncation=True)
# model(tokens)
import torch

for k,v in tokens.items():
    tokens[k] = torch.tensor(v).to('mps')
tokens.pop('token_type_ids')
output = model(tokens)



In [49]:
output['sentence_embedding'].shape
output.keys

<bound method BatchEncoding.keys of {'input_ids': tensor([[   0,    6, 4028, 4028,    2,    1,    1],
        [   0,  104, 4028, 4028,    7,   71,    2]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]], device='mps:0'), 'token_embeddings': tensor([[[-2.4007e-01,  1.2291e-01,  1.5371e-02,  ...,  1.5044e-01,
           4.8351e-02,  1.8046e-01],
         [-7.1232e-01,  1.7057e-02, -8.4404e-02,  ...,  8.8616e-02,
          -1.6265e-01,  4.9053e-01],
         [ 3.0464e-02,  9.4459e-02,  2.1534e-01,  ..., -3.3608e-04,
           3.0849e-01,  1.1893e-01],
         ...,
         [-2.4037e-01,  1.2362e-01,  1.6160e-02,  ...,  1.5071e-01,
           4.7008e-02,  1.8102e-01],
         [-1.1718e-01,  1.1169e-01,  2.1693e-02,  ...,  1.0471e-01,
           2.4098e-02,  4.7576e-01],
         [-7.5265e-02,  1.5574e-01,  4.6872e-02,  ...,  1.1901e-01,
           4.7417e-02,  4.7945e-01]],

        [[-4.0009e-01,  1.3121e-01,  1.3415e-01,  ..., -3.8679e-

In [56]:
model.state_dict()['0.auto_model.embeddings.word_embeddings.weight']

tensor([[-0.0077, -0.0007, -0.0048,  ..., -0.0184,  0.0219,  0.0019],
        [ 0.0223,  0.0060, -0.0005,  ..., -0.0176,  0.0140,  0.0122],
        [-0.0113,  0.0110,  0.0024,  ..., -0.0030,  0.0144,  0.0019],
        ...,
        [ 0.0181, -0.0058,  0.0172,  ..., -0.0180,  0.0059,  0.0012],
        [-0.0154,  0.0136, -0.0042,  ..., -0.0144,  0.0120,  0.0085],
        [-0.0016, -0.0049,  0.0032,  ..., -0.0096,  0.0114, -0.0010]],
       device='mps:0')

In [72]:
model[1](tokens)['sentence_embedding']==model[0](tokens)['sentence_embedding']

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T